In de volgende code maak ik een nieuw TIF bestand waarbij de picels met 0-waarden vervangen zijn als NoData-pixels.

In [ ]:
import numpy as np
import rasterio
from rasterio.windows import Window
import warnings
from rasterio.errors import NotGeoreferencedWarning

warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)

input_path = "../data/private/satelliet_haarlem_2024.tif"
output_path = "../data/private/satelliet_haarlem_2024_preprocessed_blocks.tif"

with rasterio.open(input_path) as src:
    profile = src.profile.copy()
    profile.update(
        dtype='float32',
        count=3,
        nodata=np.nan,
        compress='deflate',
        predictor=2,
        zlevel=6,
        bigtiff='yes'
    )

    with rasterio.open(output_path, 'w', **profile) as dst:
        # Loop over blokken
        for ji, window in src.block_windows(1):
            data = src.read([1, 2, 3], window=window)

            # Zet 0 om in NaN
            data = np.where(data == 0, np.nan, data).astype('float32')

            # Schrijf blok
            dst.write(data, window=window)

print("Grote raster verwerkt in blokken en opgeslagen als:", output_path)
